In [ ]:
import os
import datadbhandler as dbhandler
import pandas as pd
import numpy as np

PATH = 'D:/Trading/Historical Data/'
CSVPATH = 'NSE EOD/'
DBPATH = 'db/nsedb.db'
MOD_PATH = 'nse_eod_modifiers/'
AMI_PATH = 'amibroker_files/'

os.chdir(PATH)

db = dbhandler.DataDB(DBPATH)

In [ ]:
df = dbhandler.pd.read_csv(db.INDEX_CHANGE_MOD_CSV)

In [ ]:
df.head()

In [ ]:
db.fetch_records('tblModDump', [df.iloc[0][2]], start_date=df.iloc[0][1], end_date=df.iloc[0][1])

In [ ]:
db.fetch_records('tblModDump', [df.iloc[0][2]], start_date=19960918, end_date=19960918)

In [ ]:
db.truncate_table('tblDumpReplace', msg=True)

In [ ]:
db.truncate_table('tblMultipliers', msg=True)

In [ ]:
db.truncate_table('tblSkipped', msg=True)

In [ ]:
db.truncate_table('tblSymbolRange', msg=True)

In [ ]:
db.truncate_table('tblHistIndex', msg=True)

In [ ]:
db.truncate_table('tblDuplicates', msg=True)

In [ ]:
for i in range(1995, 2020):
    db.truncate_table('tblModDump{}'.format(i), msg=True)

In [ ]:
import pandas as pd

In [ ]:
df = dbhandler.pd.read_csv(db.SYMBOL_MAPPING_FILE)

In [ ]:
df2 = dbhandler.pd.read_csv(db.INDEX_CHANGE_MOD_CSV)
df2.Symbol.value_counts()

In [ ]:
df2 = dbhandler.pd.read_csv(db.INDEX_CHANGE_MOD_CSV)
df2.Symbol.value_counts()

In [ ]:
df

In [ ]:
import pandas as pd

In [ ]:
symbols_range.index = symbols_range.Symbol
symbols_range

In [ ]:
join = pd.read_sql_query(qry, db.conn)

In [ ]:
join

In [1]:
import history
import pandas as pd
import numpy as np
PATH = 'D:/Trading/Historical Data/'
DBPATH = 'db/nsedb.db'

os.chdir(PATH)

hist = history.History(DBPATH)

Opening Bhavcopy database db/nsedb.db...


In [2]:
histIndex, histIndexDates = hist.index_change_history('Nifty 50')

In [3]:
df_symbol = histIndex[histIndex.Symbol=='RELCAPITAL']
df_index_symbol = pd.merge(histIndexDates, df_symbol, how='left', on=['Date']).sort_values(by='Date')
df_index_symbol

,Date,IndexName,Symbol
0,19960918,Nifty 50,RELCAPITAL
1,19970507,Nifty 50,RELCAPITAL
2,19970514,Nifty 50,RELCAPITAL
3,19971224,Nifty 50,RELCAPITAL
4,19981007,Nifty 50,RELCAPITAL
5,19990526,NaN,NaN
6,19990908,NaN,NaN
7,20000510,NaN,NaN
8,20000524,NaN,NaN
9,20000901,NaN,NaN


In [16]:
data = hist.symbol_history(symbol='RELCAPITAL',
                           start_date='20090130',
                           end_date='20200801',
                           buffer_start='20090101',
                           index='Nifty 50')

fetching data for  RELCAPITAL


In [18]:
data[:20]

,Symbol,Date,Open,High,Low,Close,Volume,AdjustedOpen,AdjustedHigh,AdjustedLow,AdjustedClose,Buffer,IndexFlag
0,RELCAPITAL,20090101,549.9,564.85,545.00,560.55,3108852,549.9,564.85,545,560.55,1.0,NaN
1,RELCAPITAL,20090102,565.9,584.55,563.20,567.85,4685117,565.9,584.55,563.2,567.85,1.0,NaN
2,RELCAPITAL,20090105,575.0,609.75,575.00,605.60,6398972,575,609.75,575,605.6,1.0,NaN
3,RELCAPITAL,20090106,605.6,626.40,583.00,603.50,7920491,605.6,626.4,583,603.5,1.0,NaN
4,RELCAPITAL,20090107,619.9,619.90,505.05,517.50,6839760,619.9,619.9,505.05,517.5,1.0,NaN
5,RELCAPITAL,20090109,504.9,553.00,462.50,479.80,10287849,504.9,553,462.5,479.8,1.0,NaN
6,RELCAPITAL,20090112,488.7,489.90,432.55,436.95,7976127,488.7,489.9,432.55,436.95,1.0,1.0
7,RELCAPITAL,20090113,435.0,451.60,398.00,407.50,9721560,435,451.6,398,407.5,1.0,1.0
8,RELCAPITAL,20090114,419.0,441.00,412.00,433.05,6976656,419,441,412,433.05,1.0,1.0
9,RELCAPITAL,20090115,420.0,424.30,392.00,412.50,6852305,420,424.3,392,412.5,1.0,1.0


In [6]:
data.iloc[1500:1550]

,Symbol,Date,Open,High,Low,Close,Volume,AdjustedOpen,AdjustedHigh,AdjustedLow,AdjustedClose,Buffer,IndexFlag
1500,RELCAPITAL,20110822,388.40,403.70,387.00,401.75,1337532,388.4,403.7,387,401.75,NaN,1.0
1501,RELCAPITAL,20110823,406.25,410.70,399.00,403.50,1075869,406.25,410.7,399,403.5,NaN,1.0
1502,RELCAPITAL,20110824,404.95,423.80,391.00,394.35,2697525,404.95,423.8,391,394.35,NaN,1.0
1503,RELCAPITAL,20110825,398.00,399.70,385.50,391.25,1771297,398,399.7,385.5,391.25,NaN,1.0
1504,RELCAPITAL,20110826,387.40,388.00,340.10,344.20,4755497,387.4,388,340.1,344.2,NaN,1.0
1505,RELCAPITAL,20110829,350.00,390.00,349.35,377.25,5479267,350,390,349.35,377.25,NaN,1.0
1506,RELCAPITAL,20110830,390.00,396.50,371.45,384.20,5254545,390,396.5,371.45,384.2,NaN,1.0
1507,RELCAPITAL,20110902,395.70,413.70,381.25,407.60,3987195,395.7,413.7,381.25,407.6,NaN,1.0
1508,RELCAPITAL,20110905,400.05,433.50,397.00,429.55,6243360,400.05,433.5,397,429.55,NaN,1.0
1509,RELCAPITAL,20110906,426.10,432.80,411.25,423.15,3635838,426.1,432.8,411.25,423.15,NaN,1.0


In [7]:
data[:852].IndexFlag.sum()

0.0

In [8]:
data.iloc[852:1532][data.IndexFlag!=1.0]

,Symbol,Date,Open,High,Low,Close,Volume,AdjustedOpen,AdjustedHigh,AdjustedLow,AdjustedClose,Buffer,IndexFlag


In [19]:
data = hist.index_components_history('Nifty 50', start_date='19960130', end_date='20200801', buffer_start='19960101')

fetching data for  COCHINREFN
fetching data for  PONDS
fetching data for  THERMAX
fetching data for  SBIN
fetching data for  CASTROL
fetching data for  ESSARGUJ
fetching data for  IFCI
fetching data for  AMBUJACEM
fetching data for  TATACHEM
fetching data for  COLGATE
fetching data for  TATASTEEL
fetching data for  INDOGULF
fetching data for  RELIANCE
fetching data for  DRREDDY
fetching data for  IPCL
fetching data for  IDBI
fetching data for  EIHOTEL
fetching data for  HINDUNILVR
fetching data for  BAJAJHLDNG
fetching data for  ITC
fetching data for  GRASIM
fetching data for  TATACONSUM
fetching data for  SCICI
fetching data for  ABIRLANUVO
fetching data for  ARVIND
fetching data for  TATAMOTORS
fetching data for  RELPETRO
fetching data for  GESHIPPING
fetching data for  RELINFRA
fetching data for  ORIENTBANK
fetching data for  L&T
fetching data for  RANBAXY
fetching data for  NESTLE
fetching data for  RELCAPITAL
fetching data for  ICICI
fetching data for  TVSSUZUKI
fetching data for 

In [20]:
data['RELCAPITAL']

,Symbol,Date,Open,High,Low,Close,Volume,AdjustedOpen,AdjustedHigh,AdjustedLow,AdjustedClose,Buffer,IndexFlag
0,RELCAPITAL,19960101,90.30,90.9,90.00,90.70,22700,90.3,90.9,90,90.7,1.0,NaN
1,RELCAPITAL,19960102,90.70,91.0,89.75,90.15,80400,90.7,91,89.75,90.15,1.0,NaN
2,RELCAPITAL,19960109,84.50,84.7,81.00,83.60,69700,84.5,84.7,81,83.6,1.0,NaN
3,RELCAPITAL,19960110,83.60,84.5,82.50,82.95,57800,83.6,84.5,82.5,82.95,1.0,NaN
4,RELCAPITAL,19960111,82.95,85.0,82.35,84.90,92100,82.95,85,82.35,84.9,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6069,RELCAPITAL,20200727,10.40,10.4,9.95,10.00,1439878,10.4,10.4,9.95,10,NaN,NaN
6070,RELCAPITAL,20200728,10.10,10.2,9.50,9.65,1048030,10.1,10.2,9.5,9.65,NaN,NaN
6071,RELCAPITAL,20200729,9.80,10.1,9.30,10.10,653461,9.8,10.1,9.3,10.1,NaN,NaN
6072,RELCAPITAL,20200730,10.00,10.4,9.60,9.85,904157,10,10.4,9.6,9.85,NaN,NaN


In [18]:
import pandas as pd
import numpy as np
start_date = buffer_start = '19960101'
end_date = '20200801'
symbol = 'TCS'
index = 'Nifty 50'

buffer_start = hist.date_check(start_date, end_date, buffer_start)

symbol_hist = hist.symbol_history(symbol, start_date, end_date, buffer_start)

histIndex, histIndexDates = hist.index_change_history(index)

# Index change dates when symbol was part of index
df_symbol = histIndex[histIndex.Symbol == symbol]

# All index change dates with info on symbol present or absent
df_index_symbol = pd.merge(histIndexDates, df_symbol, how='left', on=['Date']).sort_values(by='Date')
df_index_symbol.Symbol = np.where(df_index_symbol.IndexName.notnull(), True, np.nan)

# Marking dates when symbol was removed from index - 1.0 = Present, 0.0 = Removal, NaN = Absent
df_index_symbol.Symbol.fillna(method='ffill', inplace=True, limit=1) 
df_index_symbol.Symbol = np.where((df_index_symbol.IndexName.isnull()) & \
    (df_index_symbol.Symbol.notnull()), False, df_index_symbol.Symbol)
# Removing NaN records, renaming columns
df_index_symbol = df_index_symbol[df_index_symbol.Symbol.notnull()][['Date', 'Symbol']]
df_index_symbol.columns = ['Date', 'IndexFlag']

# Symbol in index on buffer_start?
try:
    index_flag_buffer_start = df_index_symbol[df_index_symbol.Date.astype(int) < \
        int(buffer_start)].sort_values(by='Date', ascending=False).iloc[0].IndexFlag
except IndexError:
    index_flag_buffer_start = False


fetching data for  TCS


In [19]:
index_flag_buffer_start

False

In [9]:
len(data.keys())

129

In [10]:
data['TCS']

,Symbol,Date,Open,High,Low,Close,Volume,AdjustedOpen,AdjustedHigh,AdjustedLow,AdjustedClose,Buffer,IndexFlag
0,TCS,20040825,1198.70,1198.7,979.00,987.95,17116372,NaN,NaN,NaN,NaN,None,NaN
1,TCS,20040826,992.00,997.0,975.30,979.00,5055400,NaN,NaN,NaN,NaN,None,NaN
2,TCS,20040827,982.40,982.4,958.55,962.65,3830750,NaN,NaN,NaN,NaN,None,NaN
3,TCS,20040830,969.90,990.0,965.00,986.75,3058151,NaN,NaN,NaN,NaN,None,NaN
4,TCS,20040831,986.50,990.0,976.00,988.10,2649332,NaN,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3938,TCS,20200727,2165.00,2215.0,2163.50,2206.80,4248837,17320.0,17720.0,17308.0,17654.4,None,1.0
3939,TCS,20200728,2215.00,2320.0,2215.00,2309.75,8092458,17720.0,18560.0,17720.0,18478.0,None,1.0
3940,TCS,20200729,2292.00,2342.0,2246.05,2276.05,6090440,18336.0,18736.0,17968.4,18208.4,None,1.0
3941,TCS,20200730,2285.90,2358.0,2272.25,2279.50,7098913,18287.2,18864.0,18178.0,18236.0,None,1.0


In [5]:
data['TCS'].IndexFlag.sum()

3817.0

In [11]:
data['TCS'].iloc[100:150]

,Symbol,Date,Open,High,Low,Close,Volume,AdjustedOpen,AdjustedHigh,AdjustedLow,AdjustedClose,Buffer,IndexFlag
100,TCS,20050118,1274.70,1284.00,1257.50,1263.85,772908,NaN,NaN,NaN,NaN,None,NaN
101,TCS,20050119,1266.00,1273.70,1251.10,1255.40,442311,NaN,NaN,NaN,NaN,None,NaN
102,TCS,20050120,1255.00,1255.00,1230.00,1243.70,567025,NaN,NaN,NaN,NaN,None,NaN
103,TCS,20050124,1243.70,1247.35,1215.35,1220.65,453524,NaN,NaN,NaN,NaN,None,NaN
104,TCS,20050125,1214.80,1239.90,1207.10,1235.45,558241,NaN,NaN,NaN,NaN,None,NaN
105,TCS,20050127,1243.25,1253.80,1235.30,1240.55,681918,NaN,NaN,NaN,NaN,None,NaN
106,TCS,20050128,1242.85,1294.80,1242.50,1289.20,1049227,NaN,NaN,NaN,NaN,None,NaN
107,TCS,20050131,1298.00,1311.80,1290.25,1300.40,731673,NaN,NaN,NaN,NaN,None,NaN
108,TCS,20050201,1307.00,1320.00,1287.00,1315.25,1002480,NaN,NaN,NaN,NaN,None,NaN
109,TCS,20050202,1319.00,1323.45,1303.00,1308.95,608807,NaN,NaN,NaN,NaN,None,NaN
